# Objetivo: Note base que implementa funciones para script procesador de productos
# Autor: Gustavo V. Diaz
# Fecha: 22/09/2024

### Descripción
Script base que servirá para copiar y pegar esctructura general y funciones desarrolladas en módulo especializado para las tareas necesarias.

Se parte de la hipótesis de tener definido un dataframe que contenga la búsqueda realizada por "searcher_filt.py" junto con el kml de búsqueda inicial. Posteriormente se irá recorriendo el dataframe para:
1. Bajar producto
2. Recortarlo
3. Ingresar polígono de búsqueda a producto SNAP
4. Aplicar resize a producto SNAP
5. Enmascarar cirros
6. Enmascarar por ROI de búsqueda
7. Procesar por NDVI o RGB
8. Entregar salida según se requiera en el paso 7

#### Entrada
1. df de productos a bajar (en csv o pkl)
2. Ruta kml de búsqueda

#### Salida

1. Carpeta de salida
    1. Composiciones RGB
        1. Tiff
        1. png
    2. Tabla metadatos
        1. Productos
            1. Id
            2. acq_date
            3. Tiff_Prod_name (salida)
            4. Ruta a Tiff_Prod_name
        2. Configuración
            1. Configuración de búsqueda
            2. Cantidad de productos procesados
            3. Fecha inicio análisis
            4. Fecha fin análisis
            5. Fecha procesamiento
            6. Nombre ROI

In [1]:
# Declaración de módulos necesarios
# Para lectura de csv
import pandas as pd
import sys
import os
import configparser

# sys.path.append(r'../utils')
# import mod_search_filt as msf

# Declaración de variables para configuración
input_path = r'/src/output/research_20240928T190500_Tratayen.csv'
input_path_pkl = r'/src/output/research_20240928T190500_Tratayen.pkl'
verbose = True
kml_path = r'/src/Vectores/Tratayen.kml'

# Lectura de configurador de procesador
# Script para generar request de búsqueda y filtrado de productos Sentinel-2

# import mod_search_filt as msf

In [2]:
# Creación o lectura de archivo de configuración
def read_conf_proc(path2conf, verbose):
    if not(os.path.exists(path2conf)):
        create_conf_file_proc(path2conf)
    if verbose:
        with open(path2conf, 'r') as file:
            # Read the content of the file
            file_content = file.read()
            # Print the content
            print("File Content:\n", file_content)
    conf2dict = configparser.ConfigParser()
    with open(path2conf,"r") as file:
        conf2dict.read_file(file)
    # conf2dict.read_file(path2conf)
    return {s:dict(conf2dict.items(s)) for s in conf2dict.sections()}

def create_conf_file_proc(path2conf):
    # Configuración inicial
    # ROI*
    # Fecha de inicio de búsqueda
    # Fecha de fin de búsqueda
    # Tipo de producto Sentinel
    # Porcentaje de nubosidad límite
    # Nombre de proyecto relacionado
    dict_gen = {
        'FOLDERS': {
            ';Prueba de comentarios para FOLDERS':None,
            'ROI': r'/src/Vectores/',
            'OUTPUT': '/src/Output/'
        },
        'ATTRIB': {
            ';Prueba de comentarios para ATTRIB':None,
            'init_date':'01-01-2019',
            'final_date':'31-01-2021',
            'max_cloud':'50',
            'Sent_mission':'MSIL2A',
            'proj_name':'Your name'
        },
        'ESA_SERVER': {
            ';Prueba de comentarios para ESA_SERVER':None,
            'url':'https://catalogue.dataspace.copernicus.eu/odata/v1/Products',
            'orderby': 'ContentDate/Start',
            'top':'100'
    },
        'SCRIPTING':{
            ';Configuración para aplicar en funciones':None,
            'verbose': False
        }
    }
    with open(path2conf,"w") as file:
    # file =open("employee1.ini","w")
        config_object = configparser.ConfigParser(allow_no_value=True)
    # myDict={'employee': {'name': 'John Doe', 'age': '35'},
    #         'job': {'title': 'Software Engineer', 'department': 'IT', 'years_of_experience': '10'},
    #         'address': {'street': '123 Main St.', 'city': 'San Francisco', 'state': 'CA', 'zip': '94102'}}
        sections=dict_gen.keys()
        for section in sections:
            config_object.add_section(section)
        for section in sections:
            inner_dict=dict_gen[section]
            fields=inner_dict.keys()
            for field in fields:
                value=inner_dict[field]
                config_object.set(section, field, str(value))
        config_object.write(file)
    # file.close()
    return None

In [5]:
# Cuerpo de script a ejecutar

config_path = r'/src/utils/CONF_PROC.INI'

verbose2conf = False
# Lectura de archivo de configuración de búsqueda
conf_dict = read_conf_proc(config_path, verbose2conf)

display(conf_dict)

# Reescribo variables a partir de conf_dict recién escrito
input_path = r'/src/output/research_20240928T190500_Tratayen.csv'
print(input_path, conf_dict['FOLDERS']['prods_list'], sep='\n')
# input_path_pkl = r'/src/output/research_20240928T190500_Tratayen.pkl'
# print(input_path_pkl, conf_dict['FOLDERS']['prods_list'])
# verbose = True
kml_path = r'/src/Vectores/Tratayen.kml'
print(kml_path, conf_dict['FOLDERS']['kml_input'], sep='\n')

{'FOLDERS': {'prods_list': '/src/Output/research_20241011T211427_Tratayen.csv',
  'kml_input': '/src/Vectores/Tratayen.kml',
  'wkt_roi': '/src/Vectores/aux_wkt/wkt_file.txt',
  'output': '/src/Output/Del_Ru_20241011T211427'},
 'ATTRIB': {'user': 'gus838@gmail.com',
  'pass': 'Ul!RsPWTPuw3',
  'proj_name': 'Del_Ru'},
 'PROCESSOR': {'type': 'RGB'}}

/src/output/research_20240928T190500_Tratayen.csv
/src/Output/research_20241011T211427_Tratayen.csv
/src/Vectores/Tratayen.kml
/src/Vectores/Tratayen.kml


In [7]:
# Lectura de dataframes de búsqueda a partir del csv

def lectura_csv(path, verbose):
    df = pd.read_csv(path)
    if verbose:
        print(f'Muestro variable path de función {lectura_csv.__name__}')
        print()
        display(df)
    return df

def lectura_pkl(path, verbose):
    df = pd.read_pickle(path)
    if verbose:
        print(f'Muestro variable path de función {lectura_csv.__name__}')
        print()
        display(df)
    return df

In [8]:
df = lectura_csv(conf_dict['FOLDERS']['prods_list'], verbose)

FileNotFoundError: [Errno 2] No such file or directory: '/src/Output/research_20241011T211427_Tratayen.csv'

In [4]:
# Prueba para lectura de df en pkl, no pude leerla
# lectura_pkl(input_path_pkl, verbose)
# Error: ValueError: unsupported pickle protocol: 5

In [5]:
# Iteración sobre todos los productos
# Dejo una configuración que me permitirá solo procesar el primer producto,\
# cuando verifique que funciona dehabilitaré la función para que procese todos los productos de la misma manera que el primero
flag_one_proc = True
for row in df.iterrows():
    print(row[1]['Id'])
    print(row[1]['Name'] + '\n')
    
    if flag_one_proc == True:
        break
    

33a0f98c-5167-5a72-9fb9-0ed7fbb231bc
S2B_MSIL2A_20230107T141709_N0509_R010_T19HET_20230107T181706.SAFE

